# Проект по классификации комментариев

### Описание проекта

Требуется на основе набора данных с разметкой о токсичности правок обучить модель классифицировать комментарии пользователей интернет-магазина на позитивные и негативные для дальнейшей отправки на модерацию.

### План выполнения проекта

1. Загрузить и подготовить данные.
2. Обучить разные модели и выбрать лучшую. 
3. Оформить выводы.

### Описание данных

- *text* - текст комментария
- *toxic* — целевой признак.

# Содержание
1. [Подготовка](#1)
2. [Обучение](#2)
3. [Выводы](#3)

# 1. Подготовка <a id="1"></a>

In [1]:
#грузим библиотеки
import pandas as pd
import numpy as np
import nltk
#nltk.download('wordnet')
#nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')
#nltk.download('stopwords')
#from nltk.stem import WordNetLemmatizer
#from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords

import re

# progress bar
from tqdm import tqdm, tqdm_notebook, notebook

# instantiate
tqdm.pandas(tqdm_notebook)

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import cross_val_score

from sklearn.metrics import f1_score

from catboost import CatBoostClassifier
from catboost import Pool, cv

import lightgbm as lgb

import warnings

from sklearn.metrics import confusion_matrix

/opt/conda/lib/python3.7/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
# Читаем датасет
data_tox_comm = pd.read_csv('/datasets/toxic_comments.csv')

In [3]:
data_tox_comm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [4]:
data_tox_comm.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [5]:
data_tox_comm.groupby('toxic').count()

,text
toxic,
0,143346
1,16225


In [6]:
#Смотрим дубликаты
data_tox_comm.duplicated().sum()

0

In [7]:
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wn.ADJ,
                "N": wn.NOUN,
                "V": wn.VERB,
                "R": wn.ADV}
    return tag_dict.get(tag, wn.NOUN)

In [8]:
def LemmMe(lemmatizer,text):
    text_only_words = re.sub(r'[^a-zA-Z\' ]', ' ', text)
    text_list = nltk.word_tokenize(text_only_words)
    lemms = [lemmatizer.lemmatize(w,get_wordnet_pos(w)) for w in text_list]
    lemm_text = " ".join(lemms).lower()
    return lemm_text

In [9]:
#лемматизируем столбец text в датасете
#закомментировано для экономии времени - результаты лемматизации сохранили в файл и берем сразу из него
#lemmatizer = WordNetLemmatizer()
#data_tox_comm['lemm_text'] = data_tox_comm['text'].progress_apply(lambda x: LemmMe(lemmatizer,x))
#data_tox_comm.head()

In [10]:
#запишем в файл, чтобы при следующих прогонах проекта брать сразу из файла
#закомментировано, файл уже есть
#data_tox_comm.to_csv('data_tox_comm_lemm.csv',index_label = False)

In [11]:
#Считываем датасет с результатами лемматизации
data_tox_comm_lemm = pd.read_csv('data_tox_comm_lemm.csv')

In [12]:
data_tox_comm_lemm.head()

,text,toxic,lemm_text
0,Explanation\nWhy the edits made under my usern...,0,explanation why the edits make under my userna...
1,D'aww! He matches this background colour I'm s...,0,d'aww he match this background colour i 'm see...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man i 'm really not try to edit war it 's ...
3,"""\nMore\nI can't make any real suggestions on ...",0,more i ca n't make any real suggestion on impr...
4,"You, sir, are my hero. Any chance you remember...",0,you sir be my hero any chance you remember wha...


In [13]:
data_tox_comm_lemm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159571 entries, 0 to 159570
Data columns (total 3 columns):
text         159571 non-null object
toxic        159571 non-null int64
lemm_text    159565 non-null object
dtypes: int64(1), object(2)
memory usage: 4.9+ MB


In [14]:
#есть пропуски, смотрим
data_tox_comm_lemm.query('lemm_text != lemm_text')

,text,toxic,lemm_text
4482,1993\n\n1994\n\n1995\n\n1996\n\n1997\n\n1998\n...,0,NaN
6300,193.61.111.53 15:00,0,NaN
17311,~ \n\n68.193.147.157,0,NaN
52442,"14:53,",0,NaN
53787,92.24.199.233|92.24.199.233]],0,NaN
61758,"""\n\n 199.209.144.211 """,0,NaN


In [15]:
#удаляем пропуски
data_tox_comm_lemm.dropna(inplace=True)
data_tox_comm_lemm.reset_index(inplace=True)
data_tox_comm_lemm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159565 entries, 0 to 159564
Data columns (total 4 columns):
index        159565 non-null int64
text         159565 non-null object
toxic        159565 non-null int64
lemm_text    159565 non-null object
dtypes: int64(2), object(2)
memory usage: 4.9+ MB


In [16]:
#Возьмем выборку поменьше (5000 записей) для определения лучшей модели
data_tox_comm_lemm_5000 = data_tox_comm_lemm.sample(5000).reset_index(drop=True)

#разбиваем на признаки и целевой признак
target_5000 = data_tox_comm_lemm_5000['toxic']
features_5000 = data_tox_comm_lemm_5000['lemm_text']

#корпус
corpus_5000 = features_5000.values.astype('U')

#стоп-слова
stop_words = set(stopwords.words('english'))

#TF-IDF
count_tf_idf = TfidfVectorizer(stop_words=stop_words)
count_tf_idf.fit(corpus_5000)
tf_idf_5000 = count_tf_idf.transform(corpus_5000)

## Выводы по п.1.
1. Загрузили датасет с англоязычными комментариями размером 159571 запись
2. Лемматизировали записи с помощью Wordnet
3. Для выбора лучшей модели решили взять не весь датасет, а выборку из 5000 записей
4. Разбили выборку на признаки и целевой признак
5. Создали из признаков корпус
6. Создали матрицу cо значениями TF-IDF для корпуса с учетом стоп-слов

# 2. Обучение <a id="2"></a>

In [17]:
del data_tox_comm

In [18]:
#логистическая регрессия
model = LogisticRegression(random_state=12345,solver = 'sag', max_iter = 200)
f1 = cross_val_score(model, tf_idf_5000, target_5000, scoring='f1', cv=3).mean()
print('Логистическая регрессия')
print('F1 обучающей выборки=',f1)
print()
#F1 обучающей выборки= 0.34099946882581794

Логистическая регрессия
F1 обучающей выборки= 0.23768663951910027



In [19]:
#CatBoost
#параметры
cv_dataset = Pool(data=tf_idf_5000, 
                  label=target_5000, 
                  cat_features=None)

params = {"loss_function": "Logloss",
          "custom_metric": "F1",
          "verbose": False,
         }

In [20]:
#Определяем оптимальное количество итераций для CatBoost
#закомментировано в целях экономии времени при последующих прогонах, ранее определена оптимальное iterations: 100
#for best_it in range(20, 101, 20):
#    params["iterations"] = best_it
#    scores = cv(cv_dataset,
#                params,
#                fold_count=3, 
#                plot=False)
#    f1 = scores['test-F1-mean'].mean()    
#    print('iterations:',best_it,'f1:',f1)

#Результаты
#iterations: 20 f1: 0.2900258781334798
#iterations: 40 f1: 0.296753466292616
#iterations: 60 f1: 0.3039233083872931
#iterations: 80 f1: 0.32143640851624783
#iterations: 100 f1: 0.33954999731528096
        
params["iterations"] = 100 #f1 = 0.339

In [21]:
#LightGBM
d_train_5000 = lgb.Dataset(tf_idf_5000, label=target_5000)
params_lgb = {'metrics':None} 

In [22]:
#Функция расчета F1, поскольку у LightGBM нет такой метрики
def lgb_f1_score(preds, train_data):
    y_true = train_data.get_label()
    y_pred = np.abs(np.round(preds))
    y_pred[y_pred > 1] = 1
    return 'f1', f1_score(y_true, y_pred,average='binary'), True

In [23]:
#Подбираем оптимальное количество итераций для LightGBM
##закомментировано в целях экономии времени при последующих прогонах, ранее определено num_iterations = 30
#warnings.simplefilter("ignore")

#for best_iterations in range(10, 251, 20):
#    params_lgb['num_iterations'] = best_iterations
#    f1 = lgb.cv(params=params_lgb,
#                    train_set=d_train_5000,
#                    nfold=3,
#                    feval=lgb_f1_score)
#    print('num_iterations:',best_iterations,'f1:',np.mean(f1['f1-mean']))

#Результаты
#num_iterations: 10 f1: 0.182654539432826
#num_iterations: 30 f1: 0.37166400123305
#num_iterations: 50 f1: 0.4351030334048784
#num_iterations: 70 f1: 0.46569111005462926
#num_iterations: 90 f1: 0.48306930893816347
#num_iterations: 110 f1: 0.49437324776546593
#num_iterations: 130 f1: 0.5028250945434569
#num_iterations: 150 f1: 0.5096944593258662
#num_iterations: 170 f1: 0.5150065753023975
#num_iterations: 190 f1: 0.5195074529179533
#num_iterations: 210 f1: 0.5233638928328848
#num_iterations: 230 f1: 0.5266983585393978
#num_iterations: 250 f1: 0.5295467694404704

params_lgb['num_iterations'] = 250 #f1 = 0.529
warnings.simplefilter("default")

In [24]:
#Выбираем лучшей моделью LightGBM
#Работаем уже с полной выборкой - обучаем на трейне и проверяем на тесте
#P.S С полным датасетом работать не получается - падает ядро на шаге создания TF-DF, 
#пришлось сократить датасет до 100000 записей и работать дальше уже с сокращенной версией
data_tox_comm_lemm_ = data_tox_comm_lemm.sample(100000).reset_index(drop=True)

In [25]:
#разбиваем на признаки и целевой признак
target = data_tox_comm_lemm_['toxic']
features = data_tox_comm_lemm_['lemm_text']

#разбиваем на обучающую и тестовую выборки
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.25, random_state=12345)

In [26]:
#корпус обучающей выборки
corpus_train = features_train.values.astype('U')

#корпус тестовой выборки
corpus_test = features_test.values.astype('U')

In [27]:
del target, features,data_tox_comm_lemm,d_train_5000,cv_dataset,model,count_tf_idf,data_tox_comm_lemm_5000,target_5000,features_5000,corpus_5000,tf_idf_5000

In [28]:
#TF-IDF 
count_tf_idf = TfidfVectorizer(stop_words=stop_words)

In [29]:
#обучаем на train
count_tf_idf.fit(corpus_train)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True,
                stop_words={'a', 'about', 'above', 'after', 'again', 'against',
                            'ain', 'all', 'am', 'an', 'and', 'any', 'are',
                            'aren', "aren't", 'as', 'at', 'be', 'because',
                            'been', 'before', 'being', 'below', 'between',
                            'both', 'but', 'by', 'can', 'couldn', "couldn't", ...},
                strip_accents=None, sublinear_tf=False,
                token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
                vocabulary=None)

In [30]:
#создаем TF-IDF для train
tf_idf_train = count_tf_idf.transform(corpus_train)

In [31]:
#создаем TF-IDF для test
tf_idf_test = count_tf_idf.transform(corpus_test)

In [32]:
tf_idf_train.shape

(75000, 101397)

In [33]:
tf_idf_test.shape

(25000, 101397)

In [35]:
del corpus_train,corpus_test

In [36]:
%%time
#Обучаем модель LightGBM
clf = lgb.LGBMClassifier(objective='binary',num_iterations=250)
clf.fit(tf_idf_train, target_train)

/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


CPU times: user 4min 55s, sys: 869 ms, total: 4min 56s
Wall time: 4min 59s


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_iterations=250, num_leaves=31,
               objective='binary', random_state=None, reg_alpha=0.0,
               reg_lambda=0.0, silent=True, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)

In [37]:
%%time
#Предсказываем и замеряем (LightGBM)
pred_test = clf.predict(tf_idf_test)
f1 = f1_score(target_test, pred_test)
print('LightGBM')
print("F1 тестовой выборки =", f1)
print()

#F1 тестовой выборки = 0.764

LightGBM
F1 тестовой выборки = 0.7647317502198768

CPU times: user 7.4 s, sys: 1.34 ms, total: 7.4 s
Wall time: 7.47 s


In [38]:
#Проверка на адекватность
#константная модель для проверки адекватности - все 1
pred_const = np.ones(target_test.shape)
print("F1 константной модели:", f1_score(target_test, pred_const))

F1 константной модели: 0.18735498839907194


In [39]:
#матрица ошибок
print(confusion_matrix(target_test, pred_test))

[[22191   225]
 [  845  1739]]


In [40]:
#использовать BERT (а именно код из последнего урока) в проекте не удалось, поскольку ядро при работе кода BERT постоянно падает 
#(да и вообще падает при каждом удобном случае)

## Выводы по п.2
1. Используя выборку в 5000 записей, сравнили c помощью кросс-валидации три модели: логистическую регресию, CatBoost, LightGBM. 
2. Выбрали лучшую - LightGBM, у нее F1 лучше прочих (0.529) и она быстрее того же CatBoost.
3. Разбили полный датасет размером 159565 записей на учебный и тестовый (25%)
3. Обучили выбранную модель на учебном датасете
4. Получили предсказания на тесте.
5. Метрика модели на тесте F1 = 0.764, что соответствует требованию проекта (F1 не меньше 0.75)
6. Проверили модель на адекватность, сравнив с константной моделью с предсказаниями-единицами. F1 константной модели = 0.18, наша модель более чем адекватна.
6. Построили матрицу ошибок. Судя по ней, модель пропускает треть токсичных комментариев, оценивая их как нетоксичные.

# 3. Общие выводы<a id="3"></a>

1. Открыли и лемматизировали датасет с комментариями и оценкой их токсичности размером 159 571 записей
2. На выборке из 5000 записей обучили разные модели и выбрали лучшую - LightGBM
3. Выбранную модель обучили на учебной части полного датасета и проверили на тесте. 
4. Получили значение F1 для теста, равное 0.764, что соответствует требованиям проекта (>=0.75).
5. Подтвердили адекватность модели. 